First code block inserts events into MongoDB database
assumptions: (1) api_url https://api.opensea.io/api/v2/events/collection/cryptopunks_events
(2) Event types are not specified so "sale:" is taken since its asked for that in a later question


In [ ]:
import requests
from pymongo import MongoClient
import time
from datetime import datetime
#Question 6.1

# Function required to convert ISO date format to Unix timestamp
def iso_to_unix(iso_date):
    return int(datetime.fromisoformat(iso_date).timestamp())

# Here we have to first connect to MongoDB (After setting it up in the terminal -
# using docker pull mongo and docker run --name mongodb -d -p 27017:27017 mongo)
client = MongoClient('localhost', 27017)
db = client['opensea_db']
collection = db['cryptopunks_events']
collection.delete_many({})

# API endpoint 
collection_slug = 'cryptopunks'  # The slug for CryptoPunks
api_url = f'https://api.opensea.io/api/v2/events/collection/{collection_slug}'

# Date range considering the +- 2hour requirement 
occurred_after = '2021-08-31T22:00:00'  
occurred_before = '2021-09-30T22:00:00'  
after_timestamp = iso_to_unix(occurred_after)
before_timestamp = iso_to_unix(occurred_before)

params = {
    'after': after_timestamp,  
    'before': before_timestamp,  
    'event_type': 'sale', 
    'limit': '50',  # Max limit per request
}

headers = {
    'Accept': 'application/json',
    'X-API-KEY': 'fd22235ed7874ca4971d8b11fbcd7ff7', 
}

print("Starting data collection...")

while True:
    response = requests.get(api_url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        events = data.get('asset_events', [])
        if not events:
            print("No more events found.")
            break
        
        # Transform and insert events into MongoDB
        for event in events:
            # Convert total_price from wei to ether 
            total_price_wei = event.get('total_price')
            if total_price_wei:
                total_price_eth = float(total_price_wei) / 1e18
                event['total_price_eth'] = total_price_eth
            else:
                event['total_price_eth'] = None
            
            # Insert into MongoDB
            collection.insert_one(event)
        
        # Handle pagination
        next_page = data.get('next')
        if next_page:
            params['cursor'] = next_page
            time.sleep(1)  
        else:
            break
    elif response.status_code == 429:
        print("Rate limit exceeded.")
        time.sleep(10)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

In [21]:
# Total number of event transactions 
event_count = collection.count_documents({})

print(f"Total number of event transactions in the MongoDB collection: {event_count}")


Total number of event transactions in the MongoDB collection: 30400


In [22]:
# Aggregation to find the unique token identifiers and their maximum selling prices
most_expensive_punks = collection.aggregate([
    {
        "$match": {"payment.quantity": {"$ne": None}}  # Ensure payment.quantity exists
    },
    {
        "$group": {
            "_id": "$nft.identifier",  # Group by token ID
            "max_price": {"$max": "$payment.quantity"}  # Get the maximum price for each token ID
        }
    },
    {
        "$sort": {"max_price": -1}  
    },
    {
        "$limit": 10  
    }
])

print("Ten Most Expensive Punks:")
for punk in most_expensive_punks:
    payment_quantity_wei = punk['max_price']
    payment_quantity_eth = float(payment_quantity_wei) / 1e18 
    print(f"Token ID: {punk['_id']}, Selling Price (ETH): {payment_quantity_eth:.4f}")


Ten Most Expensive Punks:
Token ID: 6987, Selling Price (ETH): 290.0000
Token ID: 6645, Selling Price (ETH): 268.0000
Token ID: 7946, Selling Price (ETH): 230.0000
Token ID: 9223, Selling Price (ETH): 199.0000
Token ID: 7163, Selling Price (ETH): 195.0000
Token ID: 8393, Selling Price (ETH): 169.0000
Token ID: 9865, Selling Price (ETH): 144.4000
Token ID: 2408, Selling Price (ETH): 140.0000
Token ID: 5182, Selling Price (ETH): 140.0000
Token ID: 3054, Selling Price (ETH): 136.9500


In [27]:
# Number of unique cryptopunks or id's
unique_punks_count = collection.distinct('nft.identifier')

print(f"Number of unique CryptoPunks sold: {len(unique_punks_count)}")

Number of unique CryptoPunks sold: 49


In [24]:
sales_per_day = collection.aggregate([
    {
        "$match": {"event_type": "sale"} 
    },
    {
        "$group": {
            "_id": {
                "$dateToString": {"format": "%Y-%m-%d", "date": {
                    "$toDate": {
                        "$multiply": ["$event_timestamp", 1000]
                    }
                }}  
            },
            "count": {"$sum": 1}  # Count the number of sales for each day
        }
    },
    {
        "$sort": {"count": 1}  
    },
    {
        "$limit": 1  # Limit to the top 1 record
    }
])

for result in sales_per_day:
    print(f"Date with the least CryptoPunks sold: {result['_id']}, Number of sales: {result['count']}")


Date with the least CryptoPunks sold: 2021-09-29, Number of sales: 10336
